# Experiments: More Likely on Holidays

This notebook reproduces preprocessing and evaluation used in `more_likely_on_holidays.py`. It will try to load a tuned model from `models/best_holiday_model.pkl` if available; otherwise it will run a GridSearch (slower).

In [ ]:
# Imports
import sys
from pathlib import Path
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_recall_curve, roc_auc_score, confusion_matrix
# Ensure repo root is on path and load the script module by file path (works even without package __init__ files)
repo_root = Path('.').resolve().parent
script_path = repo_root / 'scripts' / 'machine_learning_scripts' / 'more_likely_on_holidays.py'
import importlib.util, types
spec = importlib.util.spec_from_file_location('more_hol_mod', script_path)
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)
load_and_prepare = mod.load_and_prepare
plt.rcParams['figure.figsize'] = (8,5)

In [ ]:
# Load prepared DataFrame (this uses the same preprocessing from the script)
data_path = str(repo_root / 'data' / 'campus_reports.csv')
df = load_and_prepare(data_path)
df.head()

In [ ]:
# Prepare features and target: analyze protests specifically
# ensure has_protest exists (created by load_and_prepare)
if 'has_protest' not in df.columns:
    df['has_protest'] = 0

# Show simple rates: how often protests happen on jewish holidays vs not
phol = df.groupby('is_jewish_holiday')['has_protest'].agg(['sum','count']).reset_index()
phol['rate'] = phol['sum'] / phol['count']
print('Protest rates by is_jewish_holiday:\n', phol)

# Features: include is_jewish_holiday so model can use it
feature_cols = [c for c in ['is_jewish_holiday','is_holiday','day_of_week','holiday_type','is_weekend','week_of_year','month','day_of_month','day_of_year','incidents_last_7d','incidents_last_30d','incidents_prev_day','incidents_ewm7'] if c in df.columns]
X = df[feature_cols]
X = pd.get_dummies(X, drop_first=True)
y = df['has_protest']

# Train/test split (same random state used in the script)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.value_counts().to_dict(), y_test.value_counts().to_dict()

In [ ]:
# Try to load saved best model first (faster). If not present, run GridSearch and save it.
model_path = Path('models/best_protest_model.pkl')
if model_path.exists():
    print('Loading saved model from', model_path)
    best = joblib.load(model_path)
else:
    print('Saved model not found — running GridSearch (this may take a while)')
    from sklearn.ensemble import RandomForestClassifier
    param_grid = {
        'class_weight': [None, 'balanced'],
        'max_depth': [None, 3, 5, 10],
        'n_estimators': [100, 300],
        'max_features': ['sqrt','log2','auto']
    }
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    base = RandomForestClassifier(n_estimators=100, random_state=42)
    gs = GridSearchCV(base, param_grid, scoring='f1', cv=cv, n_jobs=-1)
    gs.fit(X_train, y_train)
    print('Best params:', gs.best_params_)
    print('Best CV F1:', gs.best_score_)
    best = gs.best_estimator_
    model_path.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(best, model_path)
    print('Saved best model to', model_path)

# feature importances (if available)
if hasattr(best, 'feature_importances_'):
    fi = pd.Series(best.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    print('\nTop features:')
    print(fi.head(15))


In [ ]:
# Evaluate best estimator and compute PR/ROC
probs = best.predict_proba(X_test)[:,1] if hasattr(best, 'predict_proba') else best.decision_function(X_test)
precisions, recalls, thresholds = precision_recall_curve(y_test, probs)
f1s = 2*(precisions*recalls)/(precisions+recalls+1e-12)
best_idx = np.nanargmax(f1s)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
print('Best threshold by F1 (test):', best_threshold)
y_pred_thresh = (probs >= best_threshold).astype(int)
print('Accuracy:', accuracy_score(y_test, y_pred_thresh))
print(classification_report(y_test, y_pred_thresh))
print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred_thresh))
try:
    print('ROC AUC:', roc_auc_score(y_test, probs))
except Exception:
    pass

In [ ]:
# Plot Precision-Recall curve and F1 vs threshold
import matplotlib.pyplot as plt
plt.figure()
plt.plot(recalls, precisions, label='PR curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend()
plt.show()
# Plot F1 vs thresholds (align lengths safely)
if len(thresholds) > 0:
    plt.figure()
    # f1s has one more element than thresholds (from precision_recall_curve), so align by dropping the first f1 (corresponds to threshold below min)
    plt.plot(thresholds, f1s[1:], marker='o')
    plt.xlabel('Threshold')
    plt.ylabel('F1')
    plt.title('F1 vs Probability Threshold')
    plt.show()
else:
    print('No thresholds returned (not enough positive samples) - skipping threshold plot')

In [ ]:
# Display persisted metrics and plots from outputs/
from pathlib import Path
import pandas as pd
from IPython.display import Image, display
metrics_path = Path('outputs') / 'protest_jewish_holiday_metrics.csv'
if metrics_path.exists():
    metrics = pd.read_csv(metrics_path)
    print('Loaded metrics:')
    display(metrics)
else:
    print('Metrics file not found:', metrics_path)
# Display plots if present
for fn in ['protest_rate_by_jewish_holiday.png', 'pr_curve_best_holiday_model.png', 'f1_vs_threshold_best_holiday_model.png']:
    p = Path('outputs') / fn
    if p.exists():
        print('Showing', p)
        display(Image(filename=str(p)))
    else:
        print('Missing:', p)